In [ ]:
%load_ext autoreload
%autoreload 2
from lib import load, timing

YEAR = 2024
DAY = 14
TESTDATA = [
    None,
    'p=0,4 v=3,-3\np=6,3 v=-1,-3\np=10,3 v=-1,2\np=2,0 v=2,-1\np=0,0 v=1,3\np=3,0 v=-2,-2\np=7,6 v=-1,-3\np=3,0 v=-1,-2\np=9,3 v=2,3\np=7,3 v=-1,2\np=2,4 v=2,-3\np=9,5 v=-3,-3\n'
][0]
TEST = TESTDATA is None

In [ ]:
import re
import numpy as np

@timing
def prepare_data():
    data = load(YEAR, DAY, split_lines=True, test=TESTDATA if TEST else None)

    if TEST:
        shape = np.array([11, 7])
    else:
        shape = np.array([101, 103])
    
    re_robot = re.compile('p=([-0-9]+),([-0-9]+) v=([-0-9]+),([-0-9]+)')
    
    p = []
    v = []
    for row in data['split']:
        result = re_robot.search(row)
        p.append([int(result.group(i)) for i in range(1, 3)])
        v.append([int(result.group(i)) for i in range(3, 5)])
    
    initial = np.array(p)
    velocity = np.array(v)

    return shape, initial, velocity

In [ ]:
# Level 1
from collections import Counter

def get_quadrant(idx):
    #    x < mid y < mid x > mid y > mid
    quadrants = {
        (True,   True,   False,  False): 0, # top left
        (True,   False,  False,  True):  1, # bottom left
        (False,  True,   True,   False): 2, # top right
        (False,  False,  True,   True):  3} # bottom right
    return quadrants.get(tuple(list(idx)), 4)

def count_by_quadrant(shape, p):
    mid = shape // 2
    quad = np.apply_along_axis(get_quadrant, 1, np.hstack((p < mid, p > mid)))
    count = Counter(quad)
    return [count[i] for i in range(4)]


@timing
def level1(shape, initial, velocity):
    moved = (initial + velocity * 100) % shape
    count = count_by_quadrant(shape, moved)
    return np.prod(count)


shape, initial, velocity = prepare_data()
print(level1(shape, initial, velocity))

In [ ]:
# Level 2
def compactness(p):
    # Use variance of robot positions to measure compactness and try to recognise close bundlings:
    center = np.mean(p, axis=0)
    return int(np.sum((p - center) ** 2))

def draw(shape, p):
    grid = np.zeros(shape).transpose()
    for r in p:
        grid[r[1],r[0]] += 1
    print('\n'.join([''.join(['#' if c else ' ' for c in row]) for row in grid]))


@timing
def level2(shape, initial, velocity):
    p = initial
    best_c = compactness(p)
    best_t = 0
    for t in range(1, 10000):
        p = (p + velocity) % shape
        c = compactness(p)
        if c < 400000:
            if c < best_c:
                best_c = c
                best_p = p
                best_t = t
    draw(shape, best_p)
    return best_t


shape, initial, velocity = prepare_data()
print(level2(shape, initial, velocity))